In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import spearmanr

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

In [13]:
INTERACTIONS_FILE = '../cloud-data/its-cmo-darwin-magellan-workspaces-folders/WS_PMCB/BisCiT_Repository/results/current_version/v2.0c/aa/dtl-surfaceome-secretome-equal_0.1/results-with-evidence-full.txt'  # Cambiar por tu archivo
INDIVIDUAL_FILE = 'GENS_DEVELOPMENT_SCORE.csv'  # Cambiar por tu archivo

In [14]:
INTERACTION_COLUMNS = [
    'Gene 1',
    'Gene 2', 
    'AA Score',
    'Genetics',
    'Knowledge Graph Connectivity',
    'Single Cell DE',
    'Single Cell Expression',
    'Single Cell Specificity',
    'Ligand Receptor Significance',
    'SC KO, positive',
    'Credentialing',
    'Credentialing bulk RNA',
    'SC KO, Negative'
]

In [15]:
interactions_full = pd.read_csv(INTERACTIONS_FILE, sep='\t')
interactions_full.head()

,Gene 1,Gene 2,AA Score,Genetics,Knowledge Graph Connectivity,Single Cell DE,Single Cell Expression,Single Cell Specificity,Ligand Receptor Significance,"SC KO, positive",Credentialing,Credentialing bulk RNA,"SC KO, Negative","DTL, Th1-17","DTL, Th2",Immune Competition,"Single Cell Co-dysregulation, positive","Single Cell Co-dysregulation, negative","Single Cell Co-dysregulation, mixed"
0,CTLA4,IL2RA,3.750244,0.850673,0.932113,0.0,0.266357,0.230324,0.363851,0.157781,0.952958,0.0,-0.003812,NaN,NaN,0.0,0.0,0.0,0.0
1,IL2RA,TNFRSF1A,3.516827,0.723262,0.741249,0.0,0.334782,0.177688,0.462156,0.480434,0.669561,0.0,-0.072305,NaN,NaN,0.0,0.0,0.0,0.0
2,IL1B,IL2RA,3.456289,0.135396,0.980071,0.0,0.334045,0.198843,0.560824,0.522829,0.742309,0.0,-0.018029,NaN,NaN,0.0,0.0,0.0,0.0
3,B2M,IL2RA,3.382431,0.135396,0.657943,0.0,0.585600,0.239590,0.627474,0.417414,0.742309,0.0,-0.023294,NaN,NaN,NaN,0.0,0.0,0.0
4,IL10,IL2RA,3.368522,0.135396,0.967379,0.0,0.294547,0.211831,0.531710,0.485349,0.742309,0.0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0


In [16]:
interactions_df = interactions_full[INTERACTION_COLUMNS].copy()
interactions_df.head()

,Gene 1,Gene 2,AA Score,Genetics,Knowledge Graph Connectivity,Single Cell DE,Single Cell Expression,Single Cell Specificity,Ligand Receptor Significance,"SC KO, positive",Credentialing,Credentialing bulk RNA,"SC KO, Negative"
0,CTLA4,IL2RA,3.750244,0.850673,0.932113,0.0,0.266357,0.230324,0.363851,0.157781,0.952958,0.0,-0.003812
1,IL2RA,TNFRSF1A,3.516827,0.723262,0.741249,0.0,0.334782,0.177688,0.462156,0.480434,0.669561,0.0,-0.072305
2,IL1B,IL2RA,3.456289,0.135396,0.980071,0.0,0.334045,0.198843,0.560824,0.522829,0.742309,0.0,-0.018029
3,B2M,IL2RA,3.382431,0.135396,0.657943,0.0,0.585600,0.239590,0.627474,0.417414,0.742309,0.0,-0.023294
4,IL10,IL2RA,3.368522,0.135396,0.967379,0.0,0.294547,0.211831,0.531710,0.485349,0.742309,0.0,0.000000


In [17]:
column_mapping = {
    'Gene 1': 'Gene1',
    'Gene 2': 'Gene2',
    'AA Score': 'AA_Score',
    'Genetics': 'Genetics',
    'Knowledge Graph Connectivity': 'Knowledge_Graph_Connectivity',
    'Single Cell DE': 'Single_Cell_DE',
    'Single Cell Expression': 'Single_Cell_Expression',
    'Single Cell Specificity': 'Single_Cell_Specificity',
    'Ligand Receptor Significance': 'Ligand_Receptor_Significance',
    'SC KO, positive': 'SC_KO_positive',
    'Credentialing': 'Credentialing',
    'Credentialing bulk RNA': 'Credentialing_bulk_RNA',
    'SC KO, Negative': 'SC_KO_Negative'
}

# Renombrar columnas que existen
interactions_df = interactions_df.rename(columns=column_mapping)
interactions_df.head()

,Gene1,Gene2,AA_Score,Genetics,Knowledge_Graph_Connectivity,Single_Cell_DE,Single_Cell_Expression,Single_Cell_Specificity,Ligand_Receptor_Significance,SC_KO_positive,Credentialing,Credentialing_bulk_RNA,SC_KO_Negative
0,CTLA4,IL2RA,3.750244,0.850673,0.932113,0.0,0.266357,0.230324,0.363851,0.157781,0.952958,0.0,-0.003812
1,IL2RA,TNFRSF1A,3.516827,0.723262,0.741249,0.0,0.334782,0.177688,0.462156,0.480434,0.669561,0.0,-0.072305
2,IL1B,IL2RA,3.456289,0.135396,0.980071,0.0,0.334045,0.198843,0.560824,0.522829,0.742309,0.0,-0.018029
3,B2M,IL2RA,3.382431,0.135396,0.657943,0.0,0.585600,0.239590,0.627474,0.417414,0.742309,0.0,-0.023294
4,IL10,IL2RA,3.368522,0.135396,0.967379,0.0,0.294547,0.211831,0.531710,0.485349,0.742309,0.0,0.000000


## Individual Scores

In [18]:
individual_df = pd.read_csv(INDIVIDUAL_FILE, sep=',')
individual_df.head()

,Unnamed: 0,ensembl_gene_id,development_score_log_norm_immunology,companies_num_launched_immunology,development_score_log_norm,companies_num_launched,gene_symbol,gene_description
0,0,ENSG00000127318,0.010000,0.0,0.010000,0.0,IL22,interleukin 22 [Source:HGNC Symbol;Acc:HGNC:14...
1,1,ENSG00000262406,0.079300,0.0,0.079300,0.0,MMP12,matrix metallopeptidase 12 [Source:HGNC Symbol...
2,2,ENSG00000115607,0.000000,0.0,0.000000,0.0,IL18RAP,interleukin 18 receptor accessory protein [Sou...
3,3,ENSG00000164400,0.329563,0.0,0.987073,0.0,CSF2,colony stimulating factor 2 [Source:HGNC Symbo...
4,4,ENSG00000102970,0.120000,0.0,0.225600,0.0,CCL17,C-C motif chemokine ligand 17 [Source:HGNC Sym...


In [19]:
individual_df = individual_df[['ensembl_gene_id', 'gene_symbol', 'development_score_log_norm_immunology', 'companies_num_launched_immunology', 'development_score_log_norm', 'companies_num_launched' ]]

In [20]:
individual_df.head()

,ensembl_gene_id,gene_symbol,development_score_log_norm_immunology,companies_num_launched_immunology,development_score_log_norm,companies_num_launched
0,ENSG00000127318,IL22,0.010000,0.0,0.010000,0.0
1,ENSG00000262406,MMP12,0.079300,0.0,0.079300,0.0
2,ENSG00000115607,IL18RAP,0.000000,0.0,0.000000,0.0
3,ENSG00000164400,CSF2,0.329563,0.0,0.987073,0.0
4,ENSG00000102970,CCL17,0.120000,0.0,0.225600,0.0


In [22]:
individual_df['companies_num_launched_immunology'] = individual_df['companies_num_launched_immunology'].astype(int)

In [23]:
individual_df['companies_num_launched'] = individual_df['companies_num_launched'].astype(int)

In [24]:
individual_df.head()

,ensembl_gene_id,gene_symbol,development_score_log_norm_immunology,companies_num_launched_immunology,development_score_log_norm,companies_num_launched
0,ENSG00000127318,IL22,0.010000,0,0.010000,0
1,ENSG00000262406,MMP12,0.079300,0,0.079300,0
2,ENSG00000115607,IL18RAP,0.000000,0,0.000000,0
3,ENSG00000164400,CSF2,0.329563,0,0.987073,0
4,ENSG00000102970,CCL17,0.120000,0,0.225600,0


## Genes Intersection

In [25]:
genes_in_interactions = set(interactions_df['Gene1'].tolist() + interactions_df['Gene2'].tolist())
genes_in_individual = set(individual_df['gene_symbol'].tolist())
common_genes = genes_in_interactions & genes_in_individual
common_genes

{'MATN1',
 'SLC32A1',
 'PLP2',
 'LAX1',
 'OR8D4',
 'SLC52A3',
 'GPI',
 'SIGLEC1',
 'IHH',
 'SRGN',
 'VWA3A',
 'GPR45',
 'ADORA1',
 'RNPEP',
 'ABCC2',
 'MAP4',
 'OR4P4',
 'MMP24',
 'SLC22A24',
 'C1QC',
 'SHISAL1',
 'GH1',
 'CRLF2',
 'ADAMTS3',
 'SLC20A2',
 'NGEF',
 'LY6H',
 'MGAT4A',
 'OR2C1',
 'SLC16A12',
 'SORT1',
 'HS6ST1',
 'ABCC4',
 'ITIH4',
 'OTOR',
 'CNNM3',
 'CIP2A',
 'TMEM67',
 'SLC9A7',
 'FZD1',
 'C6orf15',
 'CD1A',
 'ABCC10',
 'DPEP3',
 'STAMBP',
 'LRIG2',
 'IL2RB',
 'PTGDR2',
 'CXCL5',
 'IL18R1',
 'CLDN4',
 'CRTAC1',
 'BTN2A2',
 'RECK',
 'TRH',
 'NOX1',
 'OR5AL1',
 'OR52M1',
 'TMEM131',
 'CMPK1',
 'FYN',
 'ACHE',
 'TNK2',
 'SLC1A1',
 'OR10G3',
 'MSLN',
 'TTYH3',
 'CLDN17',
 'A1BG',
 'PKP3',
 'IL24',
 'IL12B',
 'TMEM25',
 'CACNA1S',
 'OR52L2P',
 'CLCN5',
 'GPX7',
 'PRSS8',
 'CA10',
 'LRRC8B',
 'CDNF',
 'AUH',
 'SNAP23',
 'KCNQ5',
 'PTPRB',
 'BMX',
 'NAALADL1',
 'SMPD1',
 'ADAM7',
 'DHRS9',
 'PTPRN',
 'NPY5R',
 'BGN',
 'PRKCA',
 'CLEC18C',
 'OR10T2',
 'ABI1',
 'ACAN',
 'PYY',


In [26]:
def create_features_and_targets(interactions_df, individual_df):
    """
    Crear dataset completo combinando interactions y individual scores
    """
    
    print("🏗️  Creando features y targets...")
    
    # Crear lookup dict para scores individuales
    individual_lookup = {}
    for _, row in individual_df.iterrows():
        gene = row['gene_symbol']
        individual_lookup[gene] = {
            'immunology': row.get('development_score_log_norm_immunology', 0.0),
            'general': row.get('development_score_log_norm', 0.0)
        }
    
    features_list = []
    targets_list = []
    gene_pairs_list = []
    
    for _, row in interactions_df.iterrows():
        gene1 = row['Gene1']
        gene2 = row['Gene2']
        
        # Solo procesar si ambos genes tienen scores individuales
        if gene1 not in individual_lookup or gene2 not in individual_lookup:
            continue
        
        # === FEATURES ===
        features = {
            # Features de interacción directos
            'AA_Score': row.get('AA_Score', 0.0),
            'Genetics': row.get('Genetics', 0.0),
            'Knowledge_Graph_Connectivity': row.get('Knowledge_Graph_Connectivity', 0.0),
            'Credentialing': row.get('Credentialing', 0.0),
            'Single_Cell_Expression': row.get('Single_Cell_Expression', 0.0),
            'SC_KO_positive': row.get('SC_KO_positive', 0.0),
            'SC_KO_Negative': row.get('SC_KO_Negative', 0.0),
            
            # Features derivados
            'Genetics_x_Credentialing': row.get('Genetics', 0.0) * row.get('Credentialing', 0.0),
            'AA_Score_x_Genetics': row.get('AA_Score', 0.0) * row.get('Genetics', 0.0),
            'KO_Net_Effect': row.get('SC_KO_positive', 0.0) - row.get('SC_KO_Negative', 0.0),
        }
        
        # Features de genes individuales
        g1_scores = individual_lookup[gene1]
        g2_scores = individual_lookup[gene2]
        
        individual_features = {
            'gene1_immunology': g1_scores['immunology'],
            'gene2_immunology': g2_scores['immunology'],
            'gene1_general': g1_scores['general'],
            'gene2_general': g2_scores['general'],
            'genes_avg_immunology': (g1_scores['immunology'] + g2_scores['immunology']) / 2,
            'genes_avg_general': (g1_scores['general'] + g2_scores['general']) / 2,
            'genes_diff_immunology': abs(g1_scores['immunology'] - g2_scores['immunology']),
            'genes_max_immunology': max(g1_scores['immunology'], g2_scores['immunology']),
        }
        
        # Combinar features
        all_features = {**features, **individual_features}
        
        # === TARGETS ===
        targets = {
            # Target principal: promedio de scores individuales
            'pair_avg_immunology': (g1_scores['immunology'] + g2_scores['immunology']) / 2,
            'pair_avg_general': (g1_scores['general'] + g2_scores['general']) / 2,
            'pair_combined_score': (g1_scores['immunology'] + g1_scores['general'] + 
                                  g2_scores['immunology'] + g2_scores['general']) / 4,
            
            # Target sinérgico: beneficio adicional de la combinación
            'pair_synergy': ((g1_scores['immunology'] + g2_scores['immunology']) / 2) - 
                          max(g1_scores['immunology'], g2_scores['immunology']),
            
            # Target multiplicativo: ambos genes deben ser buenos
            'pair_multiplicative': g1_scores['immunology'] * g1_scores['general'] * 
                                 g2_scores['immunology'] * g2_scores['general']
        }
        
        # Solo incluir si tenemos datos válidos
        if any(v > 0 for v in targets.values()):
            features_list.append(all_features)
            targets_list.append(targets)
            gene_pairs_list.append((gene1, gene2))
    
    # Convertir a DataFrames
    features_df = pd.DataFrame(features_list)
    targets_df = pd.DataFrame(targets_list)
    
    # Limpiar datos
    features_df = features_df.fillna(0)
    targets_df = targets_df.fillna(0)
    
    print(f"✅ Dataset creado:")
    print(f"   Pares válidos: {len(features_df):,}")
    print(f"   Features: {len(features_df.columns)}")
    print(f"   Targets: {len(targets_df.columns)}")
    
    return features_df, targets_df, gene_pairs_list

In [27]:
X_df, y_df, gene_pairs = create_features_and_targets(interactions_df, individual_df)

🏗️  Creando features y targets...
✅ Dataset creado:
   Pares válidos: 4,797,516
   Features: 18
   Targets: 5


In [28]:
X_df.head()

,AA_Score,Genetics,Knowledge_Graph_Connectivity,Credentialing,Single_Cell_Expression,SC_KO_positive,SC_KO_Negative,Genetics_x_Credentialing,AA_Score_x_Genetics,KO_Net_Effect,gene1_immunology,gene2_immunology,gene1_general,gene2_general,genes_avg_immunology,genes_avg_general,genes_diff_immunology,genes_max_immunology
0,3.750244,0.850673,0.932113,0.952958,0.266357,0.157781,-0.003812,0.810655,3.190232,0.161593,0.010000,0.980545,0.998593,0.997386,0.495272,0.997989,0.970545,0.980545
1,3.516827,0.723262,0.741249,0.669561,0.334782,0.480434,-0.072305,0.484268,2.543587,0.552739,0.980545,0.258162,0.997386,0.294520,0.619354,0.645953,0.722382,0.980545
2,3.456289,0.135396,0.980071,0.742309,0.334045,0.522829,-0.018029,0.100506,0.467969,0.540858,0.750487,0.980545,0.956048,0.997386,0.865516,0.976717,0.230058,0.980545
3,3.382431,0.135396,0.657943,0.742309,0.585600,0.417414,-0.023294,0.100506,0.457969,0.440708,0.000000,0.980545,0.000000,0.997386,0.490272,0.498693,0.980545,0.980545
4,3.368522,0.135396,0.967379,0.742309,0.294547,0.485349,0.000000,0.100506,0.456085,0.485349,0.019900,0.980545,0.350306,0.997386,0.500222,0.673846,0.960645,0.980545


### Y Features

- pair_avg_inmunology = g1_scores['immunology'] + g2_scores['immunology']) / 2
- pair_avg_general = g1_scores['general'] + g2_scores['general']) / 2
- pair_avg_general = g1_scores['immunology'] + g1_scores['general'] + g2_scores['immunology'] + g2_scores['general']) / 4
- pair_synergy = ((g1_scores['immunology'] + g2_scores['immunology']) / 2) - max(g1_scores['immunology'], g2_scores['immunology'])
- pair_multiplicative = g1_scores['immunology'] * g1_scores['general'] * g2_scores['immunology'] * g2_scores['general']

In [29]:
y_df.head()

,pair_avg_immunology,pair_avg_general,pair_combined_score,pair_synergy,pair_multiplicative
0,0.495272,0.997989,0.746631,-0.485272,0.009766
1,0.619354,0.645953,0.632653,-0.361191,0.074360
2,0.865516,0.976717,0.921116,-0.115029,0.701703
3,0.490272,0.498693,0.494483,-0.490272,0.000000
4,0.500222,0.673846,0.587034,-0.480322,0.006818


## Create Training y Test datasets

In [32]:
TARGET_COLUMN = 'pair_combined_score'
X = X_df.values
y = y_df[TARGET_COLUMN].values

In [34]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [35]:
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

Shape X: (4797516, 18)
Shape y: (4797516,)


## First Model: Random Forest

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test

In [ ]:
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)

In [ ]:
## Cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

In [ ]:
results = {
    'model': model,
    'train_r2': train_r2,
    'test_r2': test_r2,
    'test_rmse': test_rmse,
    'test_mae': test_mae,
    'spearman_corr': spearman_corr,
    'cv_mean': cv_scores.mean(),
    'cv_std': cv_scores.std(),
    'y_pred_test': y_pred_test
}

### Feature importance